In [41]:
# from nltk.tag.perceptron import PerceptronTagger
# 
# corpus = postagging.eat_corpus(postagging.CORPUS)
# k = round(len(corpus) * 0.8)
# train_corpus = corpus[0:k]
# test_corpus = corpus[k:]
# perc_tagger = PerceptronTagger(load=False)
# perc_tagger.train(train_corpus)

In [42]:
# n = 0
# right = 0
# for sentence in test_corpus:
#     x = [word[0] for word in sentence]
#     y = [word[1] for word in sentence]
#     y_pred = [word[1] for word in perc_tagger.tag(x)]
#     n += len(sentence)
#     for tag_real, tag_pred in zip(y, y_pred):
#         if tag_real == tag_pred:
#             right += 1
# right / n

In [43]:
# from nltk.tag.hmm import HiddenMarkovModelTagger
# 
# hmm_tagger = HiddenMarkovModelTagger.train(train_corpus, test_sequence=test_corpus)
# 

In [44]:
# for sentence in test_corpus:
#     x = [word[0] for word in sentence]
#     y = [word[1] for word in sentence]
#     y_pred = [word[1] for word in hmm_tagger.tag(x)]
#     n += len(sentence)
#     for tag_real, tag_pred in zip(y, y_pred):
#         if tag_real == tag_pred:
#             right += 1
# right / n
# 

In [45]:
# dictionary = postagging.eat_dictionary(postagging.DICTIONARY)
# 

In [46]:
# text = []
# with open('dataset_37845_1.txt', encoding='UTF-8') as file:
#     for line in file.readlines():
#         text.append(postagging.prepare_sentence(line))
# 
# with open('output', 'w', encoding='UTF-8') as file:
#     for sentence in text:
#         res = hmm_tagger.tag([word.lower() for word in sentence])
#         output = []
#         for word, tag in zip(sentence, res):
#             if '_'.join(tag) in dictionary:
#                 lemma = dictionary['_'.join(tag)]
#             else:
#                 lemma = tag[0]
#             output.append(word + '{' + lemma + '=' + tag[1] + '}')
#         file.write(' '.join(output) + '\n')

In [47]:
import numpy as np

# M = np.array([[0, 1, 0, 1, 1],[1, 1, 1, 1, 1],[0, 1, 0, 0, 0],[0, 0, 1, 0, 1],[1, 0, 0, 0, 0]], dtype=float)
# ones = np.ones((5,5))
# MM = M.dot(ones) * M

In [48]:
# f = lambda x: 1 / x
# MM = f(MM) * M
# MM

In [49]:
# where = np.isnan(MM)
# MM[where] = 0
# MM

In [50]:
# f2 = lambda x: x * 0.85 + 0.15 / 5
# MM = f2(MM)
# MM

In [51]:
from NLP import nlpfunctions
import json
# 
# # Попробовать с добивкой символов и без лемматизации
# 
# texts = nlpfunctions.eat_json('dataset_43428_1.txt')
# with open('output', 'w', encoding='UTF-8') as file:
#     res = []
#     i = 0
#     for text in texts:
#         i += 1
#         
#         M = np.array(nlpfunctions.sentence_to_matrix(text), dtype=float)
#         vec = np.zeros(len(M))
#         vec[0] = 1
#         
#         for _ in range(100):
#             vec_n = vec.dot(M).transpose()
#             if ((vec_n - vec) ** 2).any() < 0.000001:
#                 break
#             else:
#                 vec = vec_n
#                 
#         n = 300 - len(text[0])
#         sen = [text[0]]
#         vec[0] = 0
#         
#         maxi = np.argmax(vec)
#         while n > 0:
#             vec[maxi] = 0
#             sen.append(text[maxi])
#             n = n - len(text[maxi])
#             maxi = np.argmax(vec)
#         
#         res.append(''.join(sen))
#         print('ping' + str(i))
#     json.dump(res, file, ensure_ascii=False)

In [52]:
# texts = nlpfunctions.eat_json('dataset_43428_1.txt')
# with open('output', 'w', encoding='UTF-8') as file:
#     res = []
#     for text in texts:
#         n = 300 - len(text[0])
#         sen = [text[0]]
#         i = 1
#         while n > 0 and i < len(text):
#             n = n - len(text[i])
#             sen.append(text[i])
#             i += 1
#         res.append(''.join(sen))
#     json.dump(res, file, ensure_ascii=False)

In [53]:
from sklearn.model_selection import train_test_split
import pandas as pd
# Попробовать еще:

# 3) Добавить доп. параметры
# 5) биграмы, триграмы

def read_file(path):
    with open(path, encoding="UTF-8") as file:
        return file.read().split("\n")

dataset = read_file('data\\texts_train.txt')[0:-1]
y = read_file("data\\scores_train.txt")[0:-1]
normalized_dataset = read_file('data\\normalized_dataset')[0:-1]

In [54]:
importance_dictionary = nlpfunctions.expression_scores_bigr(normalized_dataset, y, 3)

In [55]:
items = importance_dictionary.items()
important_words = [word for word, _ in sorted(items, key=lambda x: x[1], reverse=True)][0:500]
important_words[0:10]

[('пора', 'перечитывать'),
 ('раз', 'просто'),
 ('у', 'ребенок'),
 ('трилогия', 'я'),
 ('не_мочь', 'оставаться'),
 ('супер', 'супер'),
 ('и', 'лирический'),
 ('изумительный', 'книга'),
 ('отличный', 'перевод'),
 ('просто', 'гений')]

In [56]:
X = nlpfunctions.prepare_bow(normalized_dataset, important_words)
for x, text in zip(X, dataset):
    x.append(nlpfunctions.excl_feature(text))
    x.append(nlpfunctions.open_brackets_feature(text))
    x.append(nlpfunctions.close_brackets_feature(text))
X_train, X_eval, y_train, y_eval = train_test_split(X, y, random_state=0)
'Start learning'

'Start learning'

In [57]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

y_eval = np.array(y_eval, dtype=np.int8)

In [58]:
model = LinearSVC(max_iter=20000)
model.fit(X_train, y_train)
y_pred = np.array(model.predict(X_eval), dtype=np.int8)
mean_squared_error(y_eval, y_pred)

C:\Users\kafree\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


8.853

In [59]:
# test_texts = read_file('data\\dataset_40757_1.txt')[0:-1]
# test_dataset = nlpfunctions.test_texts_normalize(test_texts)
# X_test = nlpfunctions.prepare_bow(test_dataset, important_words)
# for x, text in zip(X_test, test_texts):
#     x.append(nlpfunctions.excl_feature(text))
#     x.append(nlpfunctions.open_brackets_feature(text))
#     x.append(nlpfunctions.close_brackets_feature(text))
# 

In [60]:
# y_pred = model.predict(X_test)
# with open('data\\answer.txt', 'w', encoding='UTF-8') as file:
#     for pred in y_pred:
#         file.write(str(pred) + '\n')